In [1]:
import pandas as pd
import numpy as np
import json
import random
import spacy
from tqdm import tqdm
import pickle
import plotly.graph_objects as go
from plotly.offline import iplot

In [2]:
import math
from sklearn.model_selection import train_test_split

In [3]:
from spacy.tokens import Doc

In [4]:
from spacy.training import Example

In [5]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Task description

- train NER model to extract geo, gpe, tim and nat entities
- experiment with different language model sizes
- experiment with DROPOUT and epochs to get the best test set results
- analize learning curves and performance per entity

News entities
- geo = Geographical Entity
- gpe = Geopolitical Entity
- tim = Time indicator
- nat = Natural Phenomenon

# Prepare NER data

In [10]:
with open('../data/GMB_data_spacy_geo.pickle', 'rb') as f:
     spacy_data = pickle.load(f)

In [11]:
len(spacy_data)

35177

In [12]:
spacy_data[36][1]

{'entities': []}

In [13]:
spacy_data[35000][0]

"Lebanon 's top Shi'ite cleric is opposing British Prime Minister Tony Blair 's expected visit to Beirut Monday ."

## Test train split

In [14]:
# Divide spacy into train and test sets
spacy_data_train, spacy_data_test = train_test_split(spacy_data, test_size=0.1, random_state=42)

In [15]:
spacy_data_train[1]

('U.S. health officials said Friday that trials found only a small number of cases where adverse reactions might be attributed to Nevirapine .',
 {'entities': [(0, 4, 'geo'), (27, 33, 'tim')]})

## NER model setup

In [16]:
# Setup model - experiment with sm, md, lg
nlp = spacy.load('en_core_web_sm')
ner = nlp.create_pipe('ner')



In [17]:
def split_examples_to_batches(examples, batch_size):
    batches=[]
    for i in range(0, math.ceil(len(examples)/batch_size)):
        start=i*batch_size
        end = start+batch_size
        batches.append(examples[start:end])
    return batches

In [18]:
def prepare_examples(data):
    examples = []
    for i in range(0, len(data)):
        raw_text, entity_offsets = data[i]

        try:

            doc= nlp.make_doc(raw_text.lower())
            example_test = Example.from_dict(doc, entity_offsets)
            examples.append(example_test)
           
        except Exception as e:
            print(e)
            pass
    return examples

In [19]:
# prepare examples
examples_train = prepare_examples(spacy_data_train)
examples_test = prepare_examples(spacy_data_test)

In [20]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

### Add labels

In [21]:
for _, annotations in spacy_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [22]:
ner.labels

('geo', 'gpe', 'nat', 'tim')

In [23]:
spacy_data[12050][1]

{'entities': [(17, 21, 'geo'), (24, 30, 'geo')]}

# Train model

In [24]:
DROPOUT=0.1
epochs = 10

batch_size=64

In [25]:
def train_ner(nlp, examples_train, examples_test, epochs, batch_size, dropout):
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.create_optimizer()
        scores = []

        for i in range(0, epochs):
            random.shuffle(examples_train)

            batches = split_examples_to_batches(examples_train, batch_size)
            for batch in  tqdm(batches):
                try:
                        nlp.update(batch, sgd=optimizer, drop =DROPOUT)
                except Exception as e:
                    print(e)
                    pass
            train_score = nlp.evaluate(examples_train)
            val_score = nlp.evaluate(examples_test)
            total_f = val_score['ents_f']

            scores.append({"iter":i, "val_score":val_score, "train_score":train_score})

            print(f"Iter:{i}, f_score:{round(total_f,2)}")

    return nlp, scores

In [26]:
nlp, scores = train_ner(nlp, examples_train, examples_test, epochs, batch_size, DROPOUT)

100%|██████████| 495/495 [02:22<00:00,  3.48it/s]


Iter:0, f_score:0.87


100%|██████████| 495/495 [02:09<00:00,  3.82it/s]


Iter:1, f_score:0.88


100%|██████████| 495/495 [02:10<00:00,  3.79it/s]


...

Iter:7, f_score:0.88


100%|██████████| 495/495 [03:03<00:00,  2.69it/s]


Iter:8, f_score:0.89


100%|██████████| 495/495 [02:24<00:00,  3.42it/s]


Iter:9, f_score:0.88


# Model save/load

In [27]:
def save_spacy_model(nlp, model_path):
    nlp.to_disk(f'{model_path}')
    print(f"Saved model to {model_path}")
    bytes_data = nlp.to_bytes()

    f = open(f'{model_path}/bytes_data.bin', 'wb')
    f.write(bytes_data)
    f.close()
    print(f"Saved bytes_data to f'{model_path}/bytes_data.bin'")

In [28]:
save_spacy_model(nlp, "ner_sm_do01_ep10_bs64")

Saved model to ner_sm_do01_ep10_bs64
Saved bytes_data to f'ner_sm_do01_ep10_bs64/bytes_data.bin'


In [29]:
def load_spacy_model(model_path, base_model = "en_core_web_sm"):

    nlp = spacy.load(base_model)
    file = open(f'{model_path}/bytes_data.bin',"rb")
    bytes_data = file.read()
    config = nlp.config
    lang_cls = spacy.util.get_lang_class("en")
    nlp = lang_cls.from_config(config)
    nlp = nlp.from_disk(f'{model_path}')
    
    return nlp

In [57]:
nlp2 = load_spacy_model("ner_sm_do005_ep10_bs128")

## Evaluate model

In [30]:
scores

[{'iter': 0,
  'val_score': {'token_acc': 1.0,
   'token_p': 1.0,
   'token_r': 1.0,
   'token_f': 1.0,
   'ents_p': 0.8894586894586894,
   'ents_r': 0.849600870827286,
   'ents_f': 0.8690730258884662,
   'ents_per_type': {'geo': {'p': 0.871765601217656,
     'r': 0.8202649480844969,
     'f': 0.8452315071020106},
    'gpe': {'p': 0.9389830508474576,
     'r': 0.9358108108108109,
     'f': 0.937394247038917},
    'tim': {'p': 0.8812628689087165,
     'r': 0.8447368421052631,
     'f': 0.8626133691635876},
    'nat': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
   'speed': 21377.192592405376},
  'train_score': {'token_acc': 1.0,
   'token_p': 1.0,
   'token_r': 1.0,
   'token_f': 1.0,
   'ents_p': 0.893928419258628,
   'ents_r': 0.8578700958845297,
   'ents_f': 0.8755281524834903,
   'ents_per_type': {'geo': {'p': 0.8770837774461735,
     'r': 0.8299846687646252,
     'f': 0.8528844758607824},
    'tim': {'p': 0.880630284396618,
     'r': 0.85923203839808,
     'f': 0.8697995748557545},
    'gpe': 

In [31]:
def get_metric_per_epoch(scores, score_metric):
    scores_list = []
    for item in scores:
        epoch_score ={"epoch":item["iter"],
                     "train_score":item["train_score"][f"ents_{score_metric}"],
                     "val_score":item["val_score"][f"ents_{score_metric}"]}

        scores_list.append(epoch_score)
    df_scores = pd.DataFrame(scores_list)
    return df_scores
    

In [32]:
def get_metric_per_entity(scores, score_metric):
    # Create a function to transform scores into dataframe
    
    scores_list = []
    for item in scores:
        epoch =item["iter"]
        
        for ent in item["val_score"]['ents_per_type'].keys():
            epoch_score ={
                "epoch":epoch,
                "ent":ent,
                "train_score":item["train_score"]['ents_per_type'][ent][score_metric],
                "val_score":item["val_score"]['ents_per_type'][ent][score_metric]}
            scores_list.append(epoch_score)
    df_scores = pd.DataFrame(scores_list)
    return df_scores       
                

In [33]:
df_scores = get_metric_per_entity(scores, "f")

In [34]:
df_scores

,epoch,ent,train_score,val_score
0,0,geo,0.852884,0.845232
1,0,gpe,0.940308,0.937394
2,0,tim,0.869800,0.862613
3,0,nat,0.000000,0.000000
4,1,geo,0.882966,0.858015
5,1,gpe,0.952683,0.943364
6,1,tim,0.899815,0.871725
7,1,nat,0.279070,0.000000
8,2,geo,0.894391,0.861812
9,2,gpe,0.956567,0.944020


In [35]:
data=[]
for score in ["train_score", "val_score"]:
    
    trace=go.Scatter(
                x=df_scores.epoch,
                y=df_scores[score],
                mode='lines',
                marker=dict(
                size=5
                ),
            name=score,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title=f"<b>Performance by epoch",
    
    ))
iplot(figure)

In [36]:
data=[]
for ent in df_scores.ent.unique():
    df_plot = df_scores.loc[df_scores.ent == ent]
    
    trace=go.Scatter(
                x=df_plot.epoch,
                y=df_plot.val_score,
                mode='lines',
                marker=dict(
                size=5
                ),
            name=ent,

            )
    data.append(trace)
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title=f"<b>Performance by epoch and entity type",
    
    ))
iplot(figure)

# Make predictions

In [1]:
text = '''Donald Trump mistook E Jean Carroll, the writer who accuses him of rape, for his ex-wife Marla Maples during a deposition in the case last year, excerpts released in US district court on Wednesday showed.

“That’s Marla, yeah,” Trump said, when shown a photograph. “That’s my wife.”

The mistake was corrected by a lawyer for the 76-year-old former president. But observers said it could undermine Trump’s claim he could not have attacked Carroll because she is not his “type”.

It was not the first release of excerpts from Trump’s deposition, which happened in October. Last week, Trump was shown to have claimed Carroll “said it was very sexy to be raped”.

Carroll says Trump raped her in a department store changing room in the mid-1990s. Trump denies it.

Carroll sued Trump for defamation and under the Adult Survivors Act, a New York law which allows alleged victims of historical sexual assault to bring cases within a defined timeframe.

Trump was married to Maples, the mother of his daughter Tiffany, from 1993 to 1999, between marriages to Ivana Trump, his first wife, and Melania Trump, his third and current spouse.

The photograph he thought showed Maples shows Trump in Carroll’s company in the 1990s. In his deposition, Trump said it showed a “receiving line” at an event.'''

In [33]:
doc = nlp(text)    
for ent in doc.ents:
    print(f"{ent.label_} : {ent.text}")

geo : US
tim : Wednesday
tim : October
geo : New York
tim : from 1993 to 1999
tim : 1990s


In [34]:
colmap = {'geo': '\033[93m\033[1m',
 'tim': '\033[95m\033[1m',
 'gpe': '\033[92m\033[1m',
 'nat': '\033[96m\033[1m',
 'normal': '\033[0m'}

def highlight_entities(text, ents):
    blue_bold_char = '\033[94m\033[1m'
    back_to_normal = '\033[0m'

    previous_end=0
    text_h =""
    for ent in ents["entities"]:

        start=ent[0]
        end=ent[1]
        ent_val = text[start:end]
        try:
            text_h = text_h + text[previous_end:start]+colmap[ent[2]] +ent_val+back_to_normal
        except:
            text_h = text_h + text[previous_end:start]+blue_bold_char +ent_val+back_to_normal
        previous_end = end
    text_h = text_h+text[previous_end:]
    print(text_h)



In [35]:
def text_to_entities(text, nlp):
    print(colmap['geo'] + 'GEOGRAPHICAL ENTITIY' + colmap['normal'], '||',
        colmap['tim'] + 'TIME INDICATOR' + colmap['normal'],'||',
        colmap['gpe'] + 'GEOPOLITICAL ENTITIY' + colmap['normal'],'||',
        colmap['nat'] + 'NATURAL PHENOMENON' + colmap['normal'])
    print('')
    doc = nlp(text)   
    ents = doc.ents
    entites = []
    for ent in ents:
        entites.append((ent.start_char, ent.end_char, ent.label_))
        
    output = (text, {"entities":entites})
    highlight_entities(output[0], output[1])
    return output

In [37]:
text_to_entities(text, nlp)[0][0]

GEOGRAPHICAL ENTITIY || TIME INDICATOR || GEOPOLITICAL ENTITIY || NATURAL PHENOMENON

Donald Trump mistook E Jean Carroll, the writer who accuses him of rape, for his ex-wife Marla Maples during a deposition in the case last year, excerpts released in US district court on Wednesday showed.

“That’s Marla, yeah,” Trump said, when shown a photograph. “That’s my wife.”

The mistake was corrected by a lawyer for the 76-year-old former president. But observers said it could undermine Trump’s claim he could not have attacked Carroll because she is not his “type”.

It was not the first release of excerpts from Trump’s deposition, which happened in October. Last week, Trump was shown to have claimed Carroll “said it was very sexy to be raped”.

Carroll says Trump raped her in a department store changing room in the mid-1990s. Trump denies it.

Carroll sued Trump for defamation and under the Adult Survivors Act, a New York law which allows alleged victims of historical sexual assault to bring c

'D'

In [38]:
text_2 = '''Germany is facing a backlash from allies over its reluctance to supply Leopard 2 tanks to bolster Ukraine’s fighting capacity in 
the nearly year-long war with Russia.On Friday, 50 countries agreed to provide Kyiv with billions of dollars’ worth of military hardware, 
including armoured vehicles and munitions needed to push back Russian forces.But the German defence minister, Boris Pistorius, told reporters 
t the US Ramstein airbase in Germany that despite heightened expectations, “we still cannot say when a decision will be taken, and what the 
decision will be, when it comes to the Leopard tank”.Ukraine on Saturday denounced the “global indecision” of its allies in providing 
heavy-duty modern tanks, saying “today’s indecision is killing more of our people”.“Every day of delay is the death of Ukrainians. 
Think faster,” tweeted presidential adviser Mykhailo Podolyak.Several allies echoed the Ukrainian president, Volodymyr Zelenskiy, in saying
 the tanks were essential to Ukraine’s fight with its much larger neighbour.In a joint statement – and a rare public criticism of Europe’s 
 top power – the foreign ministers of the three Baltic states of Latvia, Estonia and Lithuania said they “call on Germany to provide Leopard
   tanks to Ukraine now”.“This is needed to stop Russian aggression, help Ukraine and restore peace in Europe quickly. Germany as the leading 
   European power has special responsibility in this regard,” said the statement, tweeted by the Latvian foreign minister, Edgars Rinkēvičs.
   Berlin has been hesitant to send the Leopards or allow other countries to transfer them to Kyiv, with reports earlier in the week saying 
   it would agree to do so only if the US provided its tanks as well. Washington has said providing its Abrams tanks to Ukraine is not 
   feasible, citing difficulties in training and maintenance.But expectations had grown ahead of Friday’s Ukraine contact group meeting of 
   about 50 US-led countries at Ramstein airbase that Germany would at least agree to let other countries operating Leopards transfer them 
   to Kyiv’s army.The US senator Lindsey Graham, a Republican from South Carolina who is visiting Kyiv, called on both sides to supply the 
   machines.“To the Germans: send tanks to Ukraine because they need them. It is in your own national interest that Putin loses in Ukraine.
   “To the Biden administration: send American tanks so that others will follow our lead,” he tweeted.The pleas came as the Russian army said
     its troops had launched an offensive in Ukraine’s Zaporizhzhia region, where fighting intensified this week after several months of an 
     almost frozen front.In its daily report on Saturday, Moscow’s forces said they had carried out “offensive operations” in the region and
       claimed to have “taken more advantageous lines and positions”.'''

In [39]:
text_to_entities(text_2, nlp)[0][0]

GEOGRAPHICAL ENTITIY || TIME INDICATOR || GEOPOLITICAL ENTITIY || NATURAL PHENOMENON

Germany is facing a backlash from allies over its reluctance to supply Leopard 2 tanks to bolster Ukraine’s fighting capacity in 
the nearly year-long war with Russia.On Friday, 50 countries agreed to provide Kyiv with billions of dollars’ worth of military hardware, 
including armoured vehicles and munitions needed to push back Russian forces.But the German defence minister, Boris Pistorius, told reporters 
t the US Ramstein airbase in Germany that despite heightened expectations, “we still cannot say when a decision will be taken, and what the 
decision will be, when it comes to the Leopard tank”.Ukraine on Saturday denounced the “global indecision” of its allies in providing 
heavy-duty modern tanks, saying “today’s indecision is killing more of our people”.“Every day of delay is the death of Ukrainians. 
Think faster,” tweeted presidential adviser Mykhailo Podolyak.Several allies echoed the Ukrain

'G'

In [ ]:
def save_spacy_model(nlp, model_path):
    nlp.to_disk(f'{model_path}')
    print(f"Saved model to {model_path}")
    bytes_data = nlp.to_bytes()

    f = open(f'{model_path}/bytes_data.bin', 'wb')
    f.write(bytes_data)
    f.close()
    print(f"Saved bytes_data to f'{model_path}/bytes_data.bin'")

In [ ]:
#save_spacy_model(nlp, "ner_per_and_org_3_do_05_e_10_bs_32_md")

</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>


---
# Appendix

In [ ]:
obj = text_to_entities(text, nlp)

In [ ]:
rang = len(obj[1]['entities'])
listent = []
for i in range(rang):
    listent.append(obj[1]['entities'][i][2])

listent = pd.unique(listent)

len(listent)

# colmap = {'PERSON': '\x1b[95m\x1b[1m',
#  'DATE': '\x1b[96m\x1b[1m',
#  'GPE': '\x1b[36m\x1b[1m',
#  'ORG': '\x1b[94m\x1b[1m',
#  'ORDINAL': '\x1b[92m\x1b[1m',
#  'LAW': '\x1b[93m\x1b[1m',
#  'NORP': '\x1b[91m\x1b[1m',
#  'PRODUCT': '\x1b[90m\x1b[1m'}

# coldi = {'PURPLE' : '\033[95m\033[1m',
#    'CYAN' : '\033[96m\033[1m',
#    'DARKCYAN' : '\033[36m\033[1m',
#    'BLUE' : '\033[94m\033[1m',
#    'GREEN' : '\033[92m\033[1m',
#    'YELLOW' : '\033[93m\033[1m',
#    'RED' : '\033[91m\033[1m',
#    'GRAY' : '\033[90m\033[1m',
# }
# colors = list(coldi.values())


# colmap = dict(zip(listent, colors))
# colmap

In [66]:
colmap = {'geo': '\x1b[95m\x1b[1m',
 'tim': '\x1b[96m\x1b[1m',
 'gpe': '\x1b[36m\x1b[1m',
 'nat': '\x1b[94m\x1b[1m'
 }

def highlight_entities(text, ents):
    blue_bold_char = '\033[94m\033[1m'
    back_to_normal = '\033[0m'

    previous_end=0
    text_h =""
    for ent in ents["entities"]:

        start=ent[0]
        end=ent[1]
        ent_val = text[start:end]
        try:
            text_h = text_h + text[previous_end:start]+colmap[ent[2]] +ent_val+back_to_normal
        except:
            text_h = text_h + text[previous_end:start]+blue_bold_char +ent_val+back_to_normal
        previous_end = end
    text_h = text_h+text[previous_end:]
    print(text_h)

In [67]:
def text_to_entities(text, nlp):
    print()
    doc = nlp(text)   
    ents = doc.ents
    entites = []
    for ent in ents:
        entites.append((ent.start_char, ent.end_char, ent.label_))
        
    output = (text, {"entities":entites})
    highlight_entities(output[0], output[1])
    return output